In [3]:
#Packages
import pandas as pd
import numpy as np

import matplotlib.font_manager
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm
import matplotlib.animation

from scipy import io
import math as m 
import csv
import time

from numpy import pi, cos, sin, sign

import pylab
from mayavi import mlab
from pyquaternion import Quaternion
from scipy.stats import binned_statistic as bstat
from scipy.spatial.transform import Rotation as R
import quaternion

import seaborn as sns

contents = io.loadmat('pData_mat.mat') #pData 1,50,20000
pData=contents['pData']
pData.shape

(20, 20000, 49)

In [4]:
def readData(filename, startN, endN, maxParticles):
    NF = 20 #Number of data fields
    
    dt = np.dtype(np.float32)
    f = open(filename, "rb")

    p_data = np.zeros([NF,maxParticles,endN-startN])
    
    f.seek(0)
    for i in range(endN-startN):
        numParticles = np.fromfile(f, dtype=dt, count=1)
        time = np.fromfile(f, dtype=dt, count=1)
        temp = np.fromfile(f, dtype=dt, count=NF*int(numParticles)).reshape((int(numParticles), NF)).transpose()
        p_data[:,0:temp.shape[1],i] = temp
        del temp
    return p_data
pData = readData('output_4_2__2_10mm_Rods.dat', 1, 50, 20000)
pData.shape

(20, 20000, 49)

# Trim and Cleanup Data

- Dropping all zero values
- Dropping particles who's:
    - x-values are less than 0.15, greater than 0.9
    - y-values are greater than 0.5

In [ ]:
frame = 47
df = pd.DataFrame(data={
    'xp': pData[4,:,frame],
    'yp': pData[2,:,frame],
    'zp': pData[3,:,frame],
    'omgx': pData[8,:,frame],
    'omgz': pData[9,:,frame],
    'omgy': pData[10,:,frame],
    'SE_a': pData[11,:,frame],
    'SE_b': pData[12,:,frame],
    'SE_c': pData[13,:,frame],
    'q0': pData[16,:,frame], #Keep q0 the same
    'q1': -pData[19,:,frame], #swap q1 --> -q2
    'q2': -pData[17,:,frame], # q2 --> -q3
    'q3': -pData[18,:,frame]}) # q3 --> -q1
sns.scatterplot(x='xp', y='zp', data=df, color='b')

df = df[df.SE_a != 0]
df = df[df.omgx != 0]

# df.drop(df[df.xp < 0.15].index, inplace=True)
# df.drop(df[df.xp > 0.9].index, inplace=True)
# df.drop(df[df.zp > 0.5].index, inplace=True)


fit = np.polyfit(df.xp, df.zp, 1)+0.04
fit_fn = np.poly1d(fit)+0.04
m,b = fit
print("slope: ",m," intercept: ",b)

df.drop(df[df.zp > m*df.xp+b].index, inplace=True)
sns.scatterplot(x="xp", y="zp", data=df, color = 'g');


In [5]:
def CleanUp(frame):
    df = pd.DataFrame(data={
        'xp': pData[4,:,frame],
        'yp': pData[2,:,frame],
        'zp': pData[3,:,frame],
        'omgx': pData[8,:,frame],
        'omgz': pData[9,:,frame],
        'omgy': pData[10,:,frame],
        'a': pData[11,:,frame],
        'b': pData[12,:,frame],
        'c': pData[13,:,frame],
        'q0': pData[16,:,frame], #Keep q0 the same
        'q1': -pData[19,:,frame], #swap q1 --> -q2
        'q2': -pData[17,:,frame], # q2 --> -q3
        'q3': -pData[18,:,frame]}) # q3 --> -q1

    df = df[df.a != 0]
    df = df[df.omgx != 0]
    df = df.reset_index(drop=True)
    
    fit = np.polyfit(df.xp, df.zp, 1)+0.04
    fit_fn = np.poly1d(fit)+0.04
    m,b = fit

    df.drop(df[df.zp > m*df.xp+b].index, inplace=True)
    return df

In [ ]:
df = CleanUp(48)
df


# Orientation and Rendering

Two Particles:
- Blue disks: d=6.35mm; l=3.175mm; SE_a = 0.002
- Red rods: d=3.175mm; l=4.75mm; SE_a = 0.001



# Plot Overlay of Rotated Particle

In [34]:
# mlab.init_notebook(backend='x3d')

def drawParticle_overlay(particle_id, frame):
    
    a, b, c = pData[11,particle_id,frame], pData[12,particle_id,frame], pData[13,particle_id,frame]
    s, t = 2, 8
    q0 = pData[16, particle_id, frame]
    q1=-pData[18,particle_id,frame] #swap q1 --> -q2
    q2=-pData[17,particle_id,frame] # q2 --> -q3
    q3=-pData[19,particle_id,frame] # q3 --> -q1
    q = R.from_quat([q0,q1,q2,q3])
    M = q.as_matrix()
#     q = Quaternion(q0,q1,q2,q3)
#     M = q.rotation_matrix
    
    x_0,y_0,z_0= 0,0,0  
    
    x, y, z = np.ogrid[-2*a:2*a:100j, -2*b:2*b:100j, -2*c:2*c:100j]
    F = (abs(x/a)**s+abs(y/b)**s)**(t/s)+abs(z/c)**t-1
    F_r = ((((M[0][0]*(x-x_0)+M[1][0]*(y-y_0)+M[2][0]*(z-z_0))/a)**s) + (((M[0][1]*(x-x_0)+M[1][1]*(y-y_0)+M[2][1]*(z-z_0))/b)**s))**(t/s)+(((M[0][2]*(x-x_0)+M[1][2]*(y-y_0)+M[2][2]*(z-z_0))/c)**t)-1
    mlab.contour3d(F,contours=[0], transparent=True)
    mlab.contour3d(F_r,contours=1,color=(0.2, 0.4, 0.5))
    return mlab.show()
#     mlab.axes()
#     mlab.outline()
#     img = mlab.screenshot()
#     mlab.close()
#     return plt.imshow(img)

In [35]:
drawParticle_overlay(1,47)

# Plot Location of Specific Particles

In [ ]:
def callParticle(particle_id, frame):
    df = CleanUp(frame)
    x = df['xp'][particle_id]
    z = df['zp'][particle_id]
    sns.scatterplot(x="xp", y="zp", data=df, color = 'g');
    plt.scatter(x,z, c= 'r')
    return plt.show(), drawParticle_overlay(particle_id,frame)

callParticle(4,0)

# Render Many Particles

In [39]:
# mlab.init_notebook(backend='x3d')
bin_num = 500

df = CleanUp(48) #This inputs the data into a dataframe at a specific time (i.e. 48)
hist, xbins, zbins = np.histogram2d(df["xp"], df["zp"], bins = bin_num) 

df.drop(df[df.xp < xbins[0]].index, inplace=True)
df.drop(df[df.xp > xbins[1]].index, inplace=True)

# Calculate rotation matrix
m=[]
for i in range(len(df.xp)):
    q = R.from_quat([df.q0.iloc[i], df.q1.iloc[i], df.q2.iloc[i], df.q3.iloc[i]])
    M = q.as_matrix()
    b = M.tolist()
    m.append(b)
df['M'] = m

# Start plotting
x, y, z = np.ogrid[-0.005:0.005:100j, -0.005:0.005:100j, -0.005:0.005:100j]
t = 8
s = 2

@mlab.animate()
def anim():
    for i in range(len(df.xp)):
        # Centered at origin (Works correctly)
#         F_r = ((((df.M.iloc[i][0][0]*(x-0)+df.M.iloc[i][1][0]*(y-0)+df.M.iloc[i][2][0]*(z-0))/df.a.iloc[i])**s+((df.M.iloc[i][0][1]*(x-0)+df.M.iloc[i][1][1]*(y-0)+df.M.iloc[i][2][1]*(z-0))/df.b.iloc[i])**s)**(t/s)+((df.M.iloc[i][0][2]*(x-0)+df.M.iloc[i][1][2]*(y-0)+df.M.iloc[i][2][2]*(z-0))/df.c.iloc[i])**t)-1    

        # Using x0, y0, z0 values (Works incorrectly)
        F_r = ((((df.M.iloc[i][0][0]*abs(x-df.xp.iloc[i])+df.M.iloc[i][1][0]*abs(y-df.yp.iloc[i])+df.M.iloc[i][2][0]*abs(z-df.zp.iloc[i]))/df.a.iloc[i])**s+((df.M.iloc[i][0][1]*abs(x-df.xp.iloc[i])+df.M.iloc[i][1][1]*abs(y-df.yp.iloc[i])+df.M.iloc[i][2][1]*abs(z-df.zp.iloc[i]))/df.b.iloc[i])**s)**(t/s)+((df.M.iloc[i][0][2]*abs(x-df.xp.iloc[i])+df.M.iloc[i][1][2]*abs(y-df.yp.iloc[i])+df.M.iloc[i][2][2]*abs(z-df.zp.iloc[i]))/df.c.iloc[i])**t)-1    

        #Plot the contour
        mlab.contour3d(F_r, contours = [0])
    yield
anim()
mlab.show()


# Compare Euler Angles

In [ ]:
df = CleanUp(48)
r_z,r_y,r_x=[],[],[]
for i in range(len(df['q0'])):
    r = R.from_quat([df.q0.iloc[i], df.q1.iloc[i], df.q2.iloc[i], df.q3.iloc[i]])
    f = r.as_euler('zyx', degrees = True)
#     r = np.quaternion(df.q0.iloc[i], df.q1.iloc[i], df.q2.iloc[i], df.q3.iloc[i])
#     f = quaternion.as_euler_angles(r)
    #     f = np.rad2deg(quaternion.as_euler_angles(r))
    r_z.append(f[0])
    r_y.append(f[1])
    r_x.append(f[2])

df['r_z'] = r_z
df['r_y'] = r_y
df['r_x'] = r_x

ma = df['r_z'].max()
mi =df['r_z'].min()
print(ma, mi)

In [ ]:
frame = 47
i = 15 #particle

xp=pData[2,i,frame]
zp=pData[3,i,frame]
yp=pData[4,i,frame]

vp=pData[5,i,frame]
wp=pData[6,i,frame]
up=pData[7,i,frame]

omgx=pData[8,i,frame]
omgz=pData[9,i,frame]
omgy=pData[10,i,frame]

a=pData[11,i,frame] #Lu has this switched, but when plotting looks better like this
b=pData[12,i,frame]
c=pData[13,i,frame]

s=pData[15,i,frame] #s = 2.0, t=8.0
t=pData[14,i,frame]

q0=pData[16,i,frame]
q1=-pData[18,i,frame] #swap q1 --> -q2
q2=-pData[17,i,frame] # q2 --> -q3
q3=-pData[19,i,frame] # q3 --> -q1

r = np.quaternion(q0, q1, q2, q3)
r_z, r_y, r_x = np.rad2deg(quaternion.as_euler_angles(r))

print(r_z, r_y, r_x)
drawParticle_overlay(i,frame)

# Average Euler Angles

In [ ]:
def averageEuler_np(angle, bin_num):
    
    r_z,r_y,r_x=[],[],[]
    for i in range(len(df['q0'])):
        r = np.quaternion(df.q0.iloc[i], df.q1.iloc[i], df.q2.iloc[i], df.q3.iloc[i])
        f = np.rad2deg(quaternion.as_euler_angles(r))
        r_z.append(f[0])
        r_y.append(f[1])
        r_x.append(f[2])

    df['r_z'] = r_z
    df['r_y'] = r_y
    df['r_x'] = r_x

    SE_a_vals = df.SE_a.unique()
    SE_a_1 = df[df.SE_a == SE_a_vals[0]]
    SE_a_2 = df[df.SE_a == SE_a_vals[1]]

    #Bins and calculating average angles

    if angle == 'x':
        y_bins,bin_edges,misc = bstat(SE_a_1['xp'], SE_a_1['r_x'], statistic = 'mean', bins = bin_num)
        std, bin_edges,misc = bstat(SE_a_1['xp'], SE_a_1['r_x'], statistic = 'std', bins = bin_num)
        y_bins2,bin_edges,misc = bstat(SE_a_2['xp'], SE_a_2['r_x'], statistic = 'mean', bins = bin_num)
        std2, bin_edges,misc = bstat(SE_a_2['xp'], SE_a_2['r_x'], statistic = 'std', bins = bin_num)

    if angle == 'y':
        y_bins,bin_edges,misc = bstat(SE_a_1['xp'], SE_a_1['r_y'], statistic = 'mean', bins = bin_num)
        y_bins2,bin_edges,misc = bstat(SE_a_2['xp'], SE_a_2['r_y'], statistic = 'mean', bins = bin_num)
        std, bin_edges,misc = bstat(SE_a_1['xp'], SE_a_1['r_y'], statistic = 'std', bins = bin_num)
        std2, bin_edges,misc = bstat(SE_a_2['xp'], SE_a_2['r_y'], statistic = 'std', bins = bin_num)

    if angle == 'z':
        y_bins,bin_edges,misc = bstat(SE_a_1['xp'], SE_a_1['r_z'], statistic = 'mean', bins = bin_num)
        std, bin_edges,misc = bstat(SE_a_1['xp'], SE_a_1['r_z'], statistic = 'std', bins = bin_num)
        y_bins2,bin_edges,misc = bstat(SE_a_2['xp'], SE_a_2['r_z'], statistic = 'mean', bins = bin_num)
        std2, bin_edges,misc = bstat(SE_a_2['xp'], SE_a_2['r_z'], statistic = 'std', bins = bin_num)

    #Total Mean
    
    #Figure    
    x_bins = (bin_edges[:-1]+bin_edges[1:])/2

    fig = plt.figure()
    ax1 = fig.add_subplot(111)

    ax1.plot(x_bins, y_bins, label='First Group -- Disks', color='blue')
    ax1.plot(x_bins, y_bins2, label='Second Group -- Rods', color='orange')
    
    ax1.plot(x_bins, y_bins+std, label='Std -- Disks', color='blue', linestyle='dashed',alpha=0.3)
    ax1.plot(x_bins, y_bins-std, label='Std -- Disks', color='blue', linestyle='dashed',alpha=0.3)
    ax1.plot(x_bins, y_bins2+std2, label='Std -- Rods', color='orange', linestyle='dashed',alpha=0.3)
    ax1.plot(x_bins, y_bins2-std2, label='Std -- Rods', color='orange', linestyle='dashed',alpha=0.3)

#     pylab.plot(x_bins, y_bins)
    pylab.ylim(-180,180)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

    return plt.show()

In [ ]:
def averageEuler_sci(angle, bin_num):
    
    r_z,r_y,r_x=[],[],[]
    for i in range(len(df['q0'])):
        r = R.from_quat([df.q0.iloc[i], df.q1.iloc[i], df.q2.iloc[i], df.q3.iloc[i]])
        f = r.as_euler('zyx', degrees = True)
        r_z.append(f[0])
        r_y.append(f[1])
        r_x.append(f[2])

    df['r_z'] = r_z
    df['r_y'] = r_y
    df['r_x'] = r_x

    SE_a_vals = df.SE_a.unique()
    SE_a_1 = df[df.SE_a == SE_a_vals[0]]
    SE_a_2 = df[df.SE_a == SE_a_vals[1]]

    #Bins and calculating average angles

    if angle == 'x':
        y_bins,bin_edges,misc = bstat(SE_a_1['xp'], SE_a_1['r_x'], statistic = 'mean', bins = bin_num)
        std, bin_edges,misc = bstat(SE_a_1['xp'], SE_a_1['r_x'], statistic = 'std', bins = bin_num)
        y_bins2,bin_edges,misc = bstat(SE_a_2['xp'], SE_a_2['r_x'], statistic = 'mean', bins = bin_num)
        std2, bin_edges,misc = bstat(SE_a_2['xp'], SE_a_2['r_x'], statistic = 'std', bins = bin_num)

    if angle == 'y':
        y_bins,bin_edges,misc = bstat(SE_a_1['xp'], SE_a_1['r_y'], statistic = 'mean', bins = bin_num)
        y_bins2,bin_edges,misc = bstat(SE_a_2['xp'], SE_a_2['r_y'], statistic = 'mean', bins = bin_num)
        std, bin_edges,misc = bstat(SE_a_1['xp'], SE_a_1['r_y'], statistic = 'std', bins = bin_num)
        std2, bin_edges,misc = bstat(SE_a_2['xp'], SE_a_2['r_y'], statistic = 'std', bins = bin_num)

    if angle == 'z':
        y_bins,bin_edges,misc = bstat(SE_a_1['xp'], SE_a_1['r_z'], statistic = 'mean', bins = bin_num)
        std, bin_edges,misc = bstat(SE_a_1['xp'], SE_a_1['r_z'], statistic = 'std', bins = bin_num)
        y_bins2,bin_edges,misc = bstat(SE_a_2['xp'], SE_a_2['r_z'], statistic = 'mean', bins = bin_num)
        std2, bin_edges,misc = bstat(SE_a_2['xp'], SE_a_2['r_z'], statistic = 'std', bins = bin_num)

    #Total Mean
    
    #Figure    
    x_bins = (bin_edges[:-1]+bin_edges[1:])/2

    fig = plt.figure()
    ax1 = fig.add_subplot(111)

    ax1.plot(x_bins, y_bins, label='First Group -- Disks', color='blue')
    ax1.plot(x_bins, y_bins2, label='Second Group -- Rods', color='orange')
    
    ax1.plot(x_bins, y_bins+std, label='Std -- Disks', color='blue', linestyle='dashed',alpha=0.3)
    ax1.plot(x_bins, y_bins-std, label='Std -- Disks', color='blue', linestyle='dashed',alpha=0.3)
    ax1.plot(x_bins, y_bins2+std2, label='Std -- Rods', color='orange', linestyle='dashed',alpha=0.3)
    ax1.plot(x_bins, y_bins2-std2, label='Std -- Rods', color='orange', linestyle='dashed',alpha=0.3)

#     pylab.plot(x_bins, y_bins)
    pylab.ylim(-180,180)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

    return plt.show()

In [ ]:
averageEuler_np('z',25)
averageEuler_np('y',25)
averageEuler_np('x',25)
averageEuler_sci('z',25)
averageEuler_sci('y',25)
averageEuler_sci('x',25)

# PDF One Bin

In [ ]:
def OneKDE_np(bin_num, b, p):
    
    # Create a Dataframe and Cleanup
    q, r_angles, r_matrix = [],[],[]
    r_z, r_y, r_x = [],[],[]
    for i in range(len(df['q0'])):
        r = np.quaternion(df.q0.iloc[i], df.q1.iloc[i], df.q2.iloc[i], df.q3.iloc[i])
        f = np.rad2deg(quaternion.as_euler_angles(r))
        r_z.append(f[0])
        r_y.append(f[1])
        r_x.append(f[2])

    df['r_z'] = r_z
    df['r_y'] = r_y
    df['r_x'] = r_x

    SE_a_vals = df.SE_a.unique()
    df_d = df[df.SE_a == SE_a_vals[p]]
#     df_r = df[df.SE_a == SE_a_vals[1]]

    #Divide Dataframe into bins
    df_d = df_d.sort_values(by=['xp'])

    df_d['bin'] = pd.cut(df_d.xp, bin_num, include_lowest = True)# create bins

    group = df_d.groupby('bin')# group on bin

    dfs = [group.get_group(x) for x in group.groups]# list comprehension to split groups into list of dataframes 

    #Plot the PDFS
    fig, [[ax1, ax2, ax3], [ax4, ax5, ax6]] = plt.subplots(2, 3, figsize = (15,8))

    plt.suptitle('KDE: One bin!', color='black')
    ax1.set_xlim([-400,400])
    ax2.set_xlim([-400,400])
    ax3.set_xlim([-400,400])
    ax4.set_xlim([-400,400])
    ax5.set_xlim([-400,400])
    ax6.set_xlim([-400,400])
    # ax1.set_ylim([0,0.02])
    # ax2.set_ylim([0,0.02])
    # ax3.set_ylim([0,0.02])

  
    dfs[b]['r_z'].hist(ax=ax1)
    dfs[b]['r_y'].hist(ax=ax2)
    dfs[b]['r_x'].hist(ax=ax3)
    dfs[b]['r_z'].plot.density(ax=ax4)
    dfs[b]['r_y'].plot.density(ax=ax5)
    dfs[b]['r_x'].plot.density(ax=ax6)
    
    # sns.kdeplot(data=df_d, x="r_x")
    # df['r_x'].plot.density(label='pandas density', lw=2)
    return plt.show()

In [ ]:
def OneKDE_sci(bin_num, b, p):
    
    # Create a Dataframe and Cleanup
    q, r_angles, r_matrix = [],[],[]
    r_z, r_y, r_x = [],[],[]
    for i in range(len(df['q0'])):
        r = R.from_quat([df.q0.iloc[i], df.q1.iloc[i], df.q2.iloc[i], df.q3.iloc[i]])
        f = r.as_euler('zyx', degrees = True)
        r_z.append(f[0])
        r_y.append(f[1])
        r_x.append(f[2])

    df['r_z'] = r_z
    df['r_y'] = r_y
    df['r_x'] = r_x

    SE_a_vals = df.SE_a.unique()
    df_d = df[df.SE_a == SE_a_vals[p]]
#     df_r = df[df.SE_a == SE_a_vals[1]]

    #Divide Dataframe into bins
    df_d = df_d.sort_values(by=['xp'])

    df_d['bin'] = pd.cut(df_d.xp, bin_num, include_lowest = True)# create bins

    group = df_d.groupby('bin')# group on bin

    dfs = [group.get_group(x) for x in group.groups]# list comprehension to split groups into list of dataframes 

    #Plot the PDFS
    fig, [[ax1, ax2, ax3], [ax4, ax5, ax6]] = plt.subplots(2, 3, figsize = (15,8))

    plt.suptitle('KDE: One bin!', color='black')
    ax1.set_xlim([-400,400])
    ax2.set_xlim([-400,400])
    ax3.set_xlim([-400,400])
    ax4.set_xlim([-400,400])
    ax5.set_xlim([-400,400])
    ax6.set_xlim([-400,400])
    # ax1.set_ylim([0,0.02])
    # ax2.set_ylim([0,0.02])
    # ax3.set_ylim([0,0.02])

  
    dfs[b]['r_z'].hist(ax=ax1)
    dfs[b]['r_y'].hist(ax=ax2)
    dfs[b]['r_x'].hist(ax=ax3)
    dfs[b]['r_z'].plot.density(ax=ax4)
    dfs[b]['r_y'].plot.density(ax=ax5)
    dfs[b]['r_x'].plot.density(ax=ax6)
    
    # sns.kdeplot(data=df_d, x="r_x")
    # df['r_x'].plot.density(label='pandas density', lw=2)
    return plt.show()

In [ ]:
OneKDE_np(10,0,0)
OneKDE_sci(10,0,0)

# PDF Multiple Bins

In [ ]:
def MultipleKDE_np(bin_num):
    
    # Create a Dataframe and Cleanup
    q, r_angles, r_matrix = [],[],[]
    r_z, r_y, r_x = [],[],[]
    for i in range(len(df['q0'])):
        r = np.quaternion(df.q0.iloc[i], df.q1.iloc[i], df.q2.iloc[i], df.q3.iloc[i])
        f = np.rad2deg(quaternion.as_euler_angles(r))
        r_z.append(f[0])
        r_y.append(f[1])
        r_x.append(f[2])

    df['r_z'] = r_z
    df['r_y'] = r_y
    df['r_x'] = r_x

    SE_a_vals = df.SE_a.unique()
    df_d = df[df.SE_a == SE_a_vals[0]]
    df_r = df[df.SE_a == SE_a_vals[1]]
    dd_mean_x= df_d.r_x.mean()
    dd_mean_y= df_d.r_y.mean()
    dd_mean_z= df_d.r_z.mean()
    dr_mean_x= df_r.r_x.mean()
    dr_mean_y= df_r.r_y.mean()
    dr_mean_z= df_r.r_z.mean()
    
    #Divide Dataframe into bins
    df_d = df_d.sort_values(by=['xp'])
    df_r = df_r.sort_values(by=['xp'])

    df_d['bin'] = pd.cut(df_d.xp, bin_num, include_lowest = True)# create bins
    df_r['bin'] = pd.cut(df_r.xp, bin_num, include_lowest = True)# create bins

    group = df_d.groupby('bin')# group on bin
    group_2 = df_r.groupby('bin')# group on bin

    dfs_d = [group.get_group(x) for x in group.groups]# list comprehension to split groups into list of dataframes 
    dfs_r = [group_2.get_group(x) for x in group_2.groups]# list comprehension to split groups into list of dataframes 

    #Plot the PDFS
    fig, [[ax1, ax2, ax3],[ax4,ax5,ax6]] = plt.subplots(2, 3, figsize = (15,8))

    plt.suptitle('KDE: Multiple bins!', color='black')
    ax1.set_xlim([-400,400])
    ax2.set_xlim([-400,400])
    ax3.set_xlim([-400,400])
    ax4.set_xlim([-400,400])
    ax5.set_xlim([-400,400])
    ax6.set_xlim([-400,400])
    # ax1.set_ylim([0,0.02])
    # ax2.set_ylim([0,0.02])
    # ax3.set_ylim([0,0.02])

    for i in range(bin_num):
        dfs_d[i]['r_z'].plot.density(ax=ax1)
        dfs_d[i]['r_y'].plot.density(ax=ax2)
        dfs_d[i]['r_x'].plot.density(ax=ax3)
        dfs_r[i]['r_z'].plot.density(ax=ax4)
        dfs_r[i]['r_y'].plot.density(ax=ax5)
        dfs_r[i]['r_x'].plot.density(ax=ax6)    
        
    # fig.text(0.04, 0.5, 'common Y', va='center', rotation='vertical')
    # sns.kdeplot(data=df_d, x="r_x")
    # df['r_x'].plot.density(label='pandas density', lw=2)
    
    return plt.show(), print('The Disk Mean (zyx):', dd_mean_z, dd_mean_y, dd_mean_x, '\nThe Rod Mean (zyx):', dr_mean_z, dr_mean_y, dr_mean_z)

In [ ]:
def MultipleKDE_sci(bin_num):
    
    # Create a Dataframe and Cleanup
    q, r_angles, r_matrix = [],[],[]
    r_z, r_y, r_x = [],[],[]
    for i in range(len(df['q0'])):
        r = R.from_quat([df.q0.iloc[i], df.q1.iloc[i], df.q2.iloc[i], df.q3.iloc[i]])
        f = r.as_euler('zyx', degrees = True)
        r_z.append(f[0])
        r_y.append(f[1])
        r_x.append(f[2])

    df['r_z'] = r_z
    df['r_y'] = r_y
    df['r_x'] = r_x

    SE_a_vals = df.SE_a.unique()
    df_d = df[df.SE_a == SE_a_vals[0]]
    df_r = df[df.SE_a == SE_a_vals[1]]
    dd_mean_x= df_d.r_x.mean()
    dd_mean_y= df_d.r_y.mean()
    dd_mean_z= df_d.r_z.mean()
    dr_mean_x= df_r.r_x.mean()
    dr_mean_y= df_r.r_y.mean()
    dr_mean_z= df_r.r_z.mean()
    
    #Divide Dataframe into bins
    df_d = df_d.sort_values(by=['xp'])
    df_r = df_r.sort_values(by=['xp'])

    df_d['bin'] = pd.cut(df_d.xp, bin_num, include_lowest = True)# create bins
    df_r['bin'] = pd.cut(df_r.xp, bin_num, include_lowest = True)# create bins

    group = df_d.groupby('bin')# group on bin
    group_2 = df_r.groupby('bin')# group on bin

    dfs_d = [group.get_group(x) for x in group.groups]# list comprehension to split groups into list of dataframes 
    dfs_r = [group_2.get_group(x) for x in group_2.groups]# list comprehension to split groups into list of dataframes 

    #Plot the PDFS
    fig, [[ax1, ax2, ax3],[ax4,ax5,ax6]] = plt.subplots(2, 3, figsize = (15,8))

    plt.suptitle('KDE: Multiple bins!', color='black')
    ax1.set_xlim([-400,400])
    ax2.set_xlim([-400,400])
    ax3.set_xlim([-400,400])
    ax4.set_xlim([-400,400])
    ax5.set_xlim([-400,400])
    ax6.set_xlim([-400,400])
    # ax1.set_ylim([0,0.02])
    # ax2.set_ylim([0,0.02])
    # ax3.set_ylim([0,0.02])

    for i in range(bin_num):
        dfs_d[i]['r_z'].plot.density(ax=ax1)
        dfs_d[i]['r_y'].plot.density(ax=ax2)
        dfs_d[i]['r_x'].plot.density(ax=ax3)
        dfs_r[i]['r_z'].plot.density(ax=ax4)
        dfs_r[i]['r_y'].plot.density(ax=ax5)
        dfs_r[i]['r_x'].plot.density(ax=ax6)    
        
    # fig.text(0.04, 0.5, 'common Y', va='center', rotation='vertical')
    # sns.kdeplot(data=df_d, x="r_x")
    # df['r_x'].plot.density(label='pandas density', lw=2)
    
    return plt.show(), print('The Disk Mean (zyx):', dd_mean_z, dd_mean_y, dd_mean_x, '\nThe Rod Mean (zyx):', dr_mean_z, dr_mean_y, dr_mean_z)

In [ ]:
MultipleKDE_np(10)
MultipleKDE_sci(10)

# One Particle Rotating

Some Particles have a constant quaternion --- therefore no omega x, z, y

In [ ]:
particle=14
particle_id = 14

df = pd.DataFrame(data={
    'xp': pData[4,particle, :],
    'yp': pData[2,particle,:],
    'zp':pData[3,particle,:],
    'SE_a': pData[11,particle, :],
    'omgx': pData[8,particle,:],
    'omgz': pData[9,particle,:],
    'omgy': pData[10,particle,:],
    'q0': pData[16,particle,:],
    'q1': pData[17,particle,:],
    'q2': pData[18,particle,:],
    'q3': pData[19,particle,:]})

import numpy as np
from mayavi import mlab


x, y, z = np.ogrid[-0.009:0.009:100j, -0.005:0.005:100j, -0.005:0.005:100j]
particle_id = 14
s, t = 2, 8

@mlab.animate(delay=500)
def anim():
    for frame in range(48):
        a, b, c = pData[11,particle_id,frame], pData[12,particle_id,frame], pData[13,particle_id,frame]
        q0 = pData[16, particle_id, frame]
        q1 = pData[17, particle_id, frame]
        q2 = pData[18, particle_id, frame]
        q3 = pData[19, particle_id, frame]
        q = Quaternion(q0,q1,q2,q3)
        R = q.rotation_matrix
        
        x_0,y_0,z_0=0,0,0
        F_r = ((((R[0][0]*(x-x_0)+R[1][0]*(y-y_0)+R[2][0]*(z-z_0))/a)**s) + (((R[0][1]*(x-x_0)+R[1][1]*(y-y_0)+R[2][1]*(z-z_0))/b)**s))**(s/t)+(((R[0][2]*(x-x_0)+R[1][2]*(y-y_0)+R[2][2]*(z-z_0))/c)**t)-1
        mlab.contour3d(F_r, contours = [0], color=(0.2, 0.4, 0.5), transparent = True)
#         time.sleep(10)
#         mlab.clf()
        yield

anim()
mlab.axes()
mlab.outline()
mlab.show()
